In [91]:
library(rugarch)
library(xdcclarge)
library(RiskPortfolios)
library(rmgarch)
library(Matrix)
library(RcppCNPy)

library('matrixcalc')

In [16]:
dccnl_portfolio1 <- function(df_rolls, N, days, month_days, num_trade, root_dir){
  res_all = c()
  for(i in c(1:num_trade)){
    t1 = Sys.time()
    print(t1)
    st_sam  = (i-1) * (days + month_days) + 1
    ed_sam = st_sam + days - 1
    st_pre = ed_sam + 1
    ed_pre = st_pre + month_days - 1
    print(c(st_sam, ed_sam, st_pre, ed_pre))
    res_sam = df_rolls[st_sam:ed_sam, 2:(N+1)]
    res_pre = df_rolls[st_pre:ed_pre, 2:(N+1)]
    file_path = paste(root_dir, i, '.csv', sep='')
    cov_pred = read.csv(file_path)
    cov_pred = matrix(as.matrix(cov_pred), N, N)
    ws = optimalPortfolio(Sigma=cov_pred, 
         control=list(type='minvol', constraint='lo'))
    ws = matrix(ws)
    res_i = res_pre %*% ws
    res_all <- c(res_all, res_i)
    t2 = Sys.time()
    print(t2-t1)
  }
  return(res_all)
}
indicators <- function(all_res){
  AV = round(mean(all_res)*252*100, 4)
  SD = round(cov(all_res, all_res)**0.5*252**0.5*100, 4)
  IR = round(AV/SD, 4)
  return(c(AV, SD, IR))
}

In [4]:
data = read.csv("./df_rolls.csv", sep=",")
df_rolls <- as.matrix(data)

In [8]:
num_trade = 190
root_dir = '~/Desktop/courses_term3/Master_project/data/cov_500_dcc/'

In [17]:
ffdata = read.csv('./ffdata_11062.csv')

In [77]:

cov_ff_pre<-function(Rtns, N, n_head=21){
# multi-univarite Garch specification:
spec = ugarchspec(variance.model=list(
model = "sGARCH", garchOrder = c(1, 1), submodel = NULL, 
external.regressors = NULL, variance.targeting = FALSE), 
mean.model = list(armaOrder = c(1, 1), 
include.mean = TRUE, archm = FALSE, 
archpow = 1, arfima = FALSE, 
external.regressors = NULL, archex = FALSE), 
distribution.model = "norm", 
start.pars = list(), 
fixed.pars = list())
mspec = multispec(replicate(spec, n=5))

# fit multi-univariate  garch
fitlist = multifit(multispec=mspec, solver="hybrid",
          data = Rtns) 
ht<-sigma(fitlist)^2 
residuals<-residuals(fitlist)

# prediction diag element of variance.
forecasts = multiforecast(fitlist, data = NULL, n.ahead = 21, n.roll = 0,
out.sample = 0, external.forecasts = list(mregfor = NULL, vregfor = NULL),
cluster = NULL)
vars = sigma(forecasts)


# fit dcc-garch
cDCC <- cdcc_estimation(ini.para=c(0.01,0.50),
        ht, residuals, method='COV', ts=21)

# predict average forcast covariance matrix
cov_all = 0 
for(i in c(1:n_head)){
  corr_i = matrix(cDCC$cdcc_Rt[i, ], N, N)
  var_i = diag(vars[i, 1, ])
  cov_i = var_i%*%corr_i%*%var_i
  
  cov_all <- cov_all + cov_i
}
cov_avg = cov_all/n_head
return(cov_avg)
}

get_cov_ls<-function(ffdata){
    st_id = 6348
    sam_size = 1260
    j = 0
    ff_ls = c('Mkt.RF', 'SMB', 'HML', 'RMW', 'CMA')
    while(st_id + sam_size + 21 <= 11602){
        print(j)
        t1 = Sys.time()
        trading_i = st_id + sam_size
        ffdata_i =ffdata[ff_ls][1:trading_i, ]
        cov_ff_i = cov_ff_pre(ffdata_i, 5, 21)
        if(j == 0){cov_ff_all = cov_ff_i}
        else{cov_ff_all<- rbind(cov_ff_all, cov_ff_i)}
        st_id <- st_id + 21
        j <- j + 1
        t2 = Sys.time()
        print(t2-t1)
    }
return(cov_ff_all)}

In [78]:
ffdata = read.csv('./ffdata_11062.csv')
ff_ls = c('Mkt.RF', 'SMB', 'HML', 'RMW', 'CMA')
cov_ff_all = get_cov_ls(ffdata)

In [82]:
install.packages('/var/folders/0y/j14nddr96kj5y605dbyml8v00000gn/T//RtmpwSsIK9/downloaded_packages/RcppCNPy_0.2.10.tgz', repos = NULL, type="source")

Installing package into ‘/Users/panyiming/Library/R/3.6/library’
(as ‘lib’ is unspecified)



In [81]:
install.packages('RcppCNPy')

Installing package into ‘/Users/panyiming/Library/R/3.6/library’
(as ‘lib’ is unspecified)




The downloaded binary packages are in
	/var/folders/0y/j14nddr96kj5y605dbyml8v00000gn/T//RtmpwSsIK9/downloaded_packages


In [ ]:
dccnl_resid <- function(resid_matrix, N, num_trade, root_dir){
  res_all = c()
  for(i in c(1:num_trade)){
    print(i)
    t1 = Sys.time()
    st_sam  = (i-1) * 500 + 1
    ed_sam = st_sam + N
    resid_sam = t(resid_matrix[st_sam:ed_sam, ])
    cov_pred = dcc_cov_pre(resid_sam, N)
    file_path = paste(root_dir, i, '.csv', sep='')
    write.table(cov_pred,  file=file_path, row.names = TRUE, col.names =TRUE, sep=',')
    t2 = Sys.time()
    print(t2-t1)
  }}

In [90]:
npySave('test_r_save.npy', fmat[1:10, 1:10])

In [94]:
fmat <- npyLoad("./ff_model_data/residual_mat5.npy")

In [95]:
resid5_matrix <- npyLoad("./ff_model_data/residual_mat5.npy")

ERROR: Error in npyLoad("~/Desktop/courses_term3/Master_project/data/ff_model_data/residual_mat5.npy"): npy_load: Error! Unable to open file ~/Desktop/courses_term3/Master_project/data/ff_model_data/residual_mat5.npy!



In [88]:
t(fmat[1:5, 1:10])

0.13664687,-0.011028896,-0.003603248,-0.024533859,0.006873757
-0.04365148,0.020558881,0.012060445,0.027779218,0.011049798
0.13841098,-0.009502676,-0.004995384,0.013543367,0.004697045
0.05350878,-0.006508135,-0.016111649,0.001207809,-0.027946927
0.08082129,-0.002064943,0.018064679,0.014717316,-0.006707783
-0.12214871,-0.033854025,-0.008873304,0.021177930,0.030397385
0.05517102,-0.018084054,-0.010894574,0.019767247,-0.001996698
-0.04576020,-0.022319559,-0.033815229,-0.018377843,0.011878661
0.17102992,0.024872571,-0.006615907,0.004274757,-0.015691129
0.12453482,0.033971324,0.014400697,0.034750861,0.015521393


In [ ]:
last_num = 11602
sam_size = 1260
st_id = 6348
while(st_id + sam_size + 21 <= last_num){
    trading_i = st_id + 1260
    st_id <- st_id + 21
    print(trading_i)
}